# Introduction
Welcome to Lab 2, in this lab we will take our first steps into applying ML to satellite data, using landcover analysis as our primary goal.

As covered in Lecture 2, landcover classification is the quintessential remote sensing task in the civil sector. The LandSat programme was originally developed in large part to enable the USA to monitor landcover more easily for a range of agricultral, govermental and finacial problemsets. It remains a primary task for governments and companies around the world.

This is a pixel-wise ML task. We will apply two methods, both of which are available in GEE as ready-to-go algorithms. In this instance we are happy to go with built-in as it allows us to become familiar with an end-to-end GEE ML workflow without getting bogged down in technical detail. No need to re-invent the wheel unless you are going to the [moon](https://www.nasa.gov/feature/ames/artemis-moon-rover-s-wheels-are-ready-to-roll/).

Remember that this is a set of exercises and tutorials that escalate in difficulty. Always show your working and do not neccesarily expect to get full marks on everything.

Use the resources available to you outside of this material. I expect you to search for concepts or terms you do not understand and to look at the documentation of the code packages that we are using in the first instance of getting stuck.
_______________________________________________________________________________

# Set up

In [1]:
# Install libs if needed, note the --quiet option means a lot less annoying info than as Lab 1 when we did the same!
# !pip install geemap --quiet
# !pip install geopandas --quiet
# !pip install scikit-learn --quiet

In [2]:
if 'google.colab' in str(get_ipython()):
    from google.colab import userdata
    EE_PROJECT_ID = userdata.get('EE_PROJECT_ID') 
else:
    from dotenv import load_dotenv
    import os
    load_dotenv()  # take environment variables
    EE_PROJECT_ID = os.getenv('EE_PROJECT_ID')

# Set up GEE API
import ee
ee.Authenticate()
ee.Initialize(project=EE_PROJECT_ID) #<- Remember to change this to your own project's name!

In [3]:
# Import other libs
import geemap
import geopandas as gpd
import pandas as pd

from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd

# Define Area of Interest (AOI)

In [4]:
# Define area of interest (e.g., Wellington, NZ)
aoi = ee.Geometry.Rectangle([174.6, -41.4, 174.9, -41.2])
Map = geemap.Map(center=[-41.3, 174.75], zoom=10)
Map.addLayer(aoi, {}, 'AOI')

# Visualize on the map to check got it right... (always a good idea)
Map

Map(center=[-41.3, 174.75], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright…

# Load and Cloud Mask Imagery from Earth Engine

In [5]:
# Set up your filter Sentinel-2 imagery
# A function that masks clouds in your S2 images via QA band
def mask_s2_clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
           qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000).select(['B2', 'B3', 'B4', 'B8'])

# Now build the clean collection with selected bands
s2 = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
      .filterBounds(aoi)
      .filterDate('2021-01-01', '2021-12-31')
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
      .map(mask_s2_clouds)
      .median() #<- think about this step and what it is doing to your image when you build your single composite from the image collection
      .clip(aoi))

(1) What other mathematical [compositing](https://developers.google.com/earth-engine/guides/ic_composite_mosaic) approaches could you take when building your single image from the filtered image collection you create in the cell above? Hint: you currently are doing a 'median-composite'. (3 pts)

In [6]:
# Set up the visualisation and then once again check it has worked, stacking layers in a sensible order
Map = geemap.Map(center=[-41.3, 174.75], zoom=11) #<- reset the map object to clear out other layers from earlier cells
Map.addLayer(s2, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}, 'S2 RGB')
Map.addLayer(aoi, {'color': 'red'}, 'AOI')
Map

Map(center=[-41.3, 174.75], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright…

________________________________________________________________________________

# Training data

We are going to train our data using the European Space Agency (ESA) world landcover map. Given that this is derived from satellite data our logic is at risk of becoming a little circular here. But as a training exercise it is a good way to have easy to access training data of the right size and type to apply to our image.

There are a wide range of landcover datasets in the GEE catalogue that you can use for this purpose:
*   https://developers.google.com/earth-engine/datasets/tags/landuse-landcover

First, we will load in the landcover map we will use to create our training dataset (https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v100):



Next we need to use our landcover map to [sample](https://developers.google.com/earth-engine/apidocs/ee-image-sample) the Sentinel 2 image, associating each pixel of a given landcover with the spectral band values 'beneath' it.

In [7]:
# Define valid WorldCover classes (10 to 100, spaced by 10)
valid_classes = ee.List.sequence(10, 100, 10) # 100 is included
# list(range(10, 100, 10)) # 100 in not included

# Need to change this to sequential list to avoid the algs thinking there are 99 classes rather than 9
remap_to = ee.List.sequence(1, 10)

# Load and mask WorldCover map to valid classes only
landcover = ee.Image('ESA/WorldCover/v100/2020').select('Map').clip(aoi)
landcover_masked = landcover.updateMask(
    landcover.remap(valid_classes, ee.List.repeat(1, 10)))

# Remap labels: create a new band with remapped values (10 → 1, ..., 100 → 10)
landcover_remapped = landcover.remap(valid_classes, remap_to).rename('Map_remapped')

(2) What year of data was this landcover map (accessed in the cell above) created on? (1 pt)

2020

In [8]:
landcover.getInfo()  # Get info about the landcover image

# Get version number (ingestion timestamp as microseconds since Unix epoch).
version = landcover.get('system:version')
display('Version:', version)  # ee.Number
display(
    'Version (as ingestion date):',
    ee.Date(ee.Number(version).divide(1000)).format(),
)

# Get the timestamp.
display('Timestamp:', ee.Date(landcover.get('system:time_start')))

'Version:'

'Version (as ingestion date):'

'Timestamp:'

In [9]:
# Add landcover original and remapped as bands to Sentinel-2 image
training_data = s2.addBands(landcover_masked.rename('Map')).addBands(landcover_remapped)

Let's inspect the resulting training dataset and make sure that you understand what you have built here.

We are going to:
- Print the first few rows (client-side).
- Check the number of samples per class (server-side with reduceColumns).

This may take a little time to run as we request data both client and server side.

In [10]:
# Sample the image
bands = ['B2', 'B3', 'B4', 'B8']
sample = training_data.select(bands + ['Map_remapped']).sample(
    region=aoi,
    scale=10,
    numPixels=5000,
    seed=2,
    geometries=True)

# Print first 10 sample points (client-side)
first_10 = sample.limit(10).getInfo()

print('\nFirst 10 training samples:')
for i, feature in enumerate(first_10['features']):
    props = feature['properties']
    print(f"Sample {i+1}: Class={props['Map_remapped']}, B2={props['B2']}, B3={props['B3']}, B4={props['B4']}, B8={props['B8']}")

# Count number of samples per class (server-side)
class_counts = sample.reduceColumns(
    reducer=ee.Reducer.frequencyHistogram(),
    selectors=['Map_remapped'])

print('\nClass distribution in sample:')
print(class_counts.getInfo())



First 10 training samples:
Sample 1: Class=8, B2=0.03869999945163727, B3=0.026639999821782112, B4=0.0066999997943639755, B8=0.003700000001117587
Sample 2: Class=1, B2=0.018300000578165054, B3=0.028699999675154686, B4=0.019099999219179153, B8=0.188400000333786
Sample 3: Class=1, B2=0.01489999983459711, B3=0.049400001764297485, B4=0.019200000911951065, B8=0.3569999933242798
Sample 4: Class=8, B2=0.021449999883770943, B3=0.026249999180436134, B4=0.008249999955296516, B8=0.0026499999221414328
Sample 5: Class=8, B2=0.028750000521540642, B3=0.028349999338388443, B4=0.006550000049173832, B8=0.002899999963119626
Sample 6: Class=3, B2=0.02288571372628212, B3=0.05414000153541565, B4=0.03181666508316994, B8=0.34104999899864197
Sample 7: Class=3, B2=0.048225000500679016, B3=0.08736667037010193, B4=0.06499999761581421, B8=0.45969998836517334
Sample 8: Class=8, B2=0.022749999538064003, B3=0.025800000876188278, B4=0.007849999703466892, B8=0.00215000007301569
Sample 9: Class=1, B2=0.03929999843239784

(3) What type of landcover is being sampled in sample number 2 of the dataset? Make sure your random seed is set to 2 in the sampling code so that you get the same answer as me. (1 pt)

Sample number 2 is `Class 1` => `Tree cover`

https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v100#bands

We are now ready to start machine learning on classifying the landcover of Wellington!
________________________________________________________________________________

# Random Forest Classification for Landcover
Let us do machine learning properly and first apply a 70/20/10 split that allows us to produce some accuracy statistics.

In [11]:
# Add random column
sample = sample.randomColumn('random')

# Split
train = sample.filter(ee.Filter.lt('random', 0.7))
valid = sample.filter(ee.Filter.And(ee.Filter.gte('random', 0.7), ee.Filter.lt('random', 0.9)))
test = sample.filter(ee.Filter.gte('random', 0.9))

Our next task is to set up a classifier object, followed by applying it to the image using the .classify operator. This operator hides a LOT of behind the scenes plumbing from Google.

We use the [GEE Random Forest](https://developers.google.com/earth-engine/apidocs/ee-classifier-smilerandomforest) model that they have pre-built for us. Specifying the number of trees to use is the only variable we must set. Othe variables are available, read the docs!

In [12]:
# Train the RF model
classifier = ee.Classifier.smileRandomForest(numberOfTrees=100).train(
    features=train,
    classProperty='Map_remapped',
    inputProperties=bands)

With the model trained, we just apply it simply using the '.classify' operator.

In [13]:
# Classify image
classified = s2.select(bands).classify(classifier)

In [14]:
classified

Let's visualize what we have just made!

In [15]:
# Classified image viz, fresh set up again to make sure we have exactly what we want displayed.
Map = geemap.Map()
Map.centerObject(aoi, 10)
Map.addLayer(classified.randomVisualizer(), {}, 'Classified')
Map.addLayer(landcover_remapped.randomVisualizer(), {} ,'ESA Landcover')
Map

Map(center=[-41.300046246343555, 174.75000000000097], controls=(WidgetControl(options=['position', 'transparen…

(4) Look at the ESA landcover map and our resulting random forest classified Sentinel 2 image map. How are they different and what might be some causes of this difference? (5 pts)

=> Model cannot do 100% accuracy

Finally, we will apply our test data split and produce the summary statistics that we need to make smart choices about our training approach.

In [16]:
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd


# Validate
validated = valid.classify(classifier, 'predicted')

# Function to export data for confusion matrix
def fc_to_lists(fc, classProp, predProp):
    values = fc.aggregate_array(classProp).getInfo()
    preds = fc.aggregate_array(predProp).getInfo()
    return values, preds

# Get predicted vs actual from validation set
y_true, y_pred = fc_to_lists(validated, 'Map_remapped', 'predicted')

# Labels for original classes
label_map = {i + 1: valid_classes.get(i).getInfo() for i in range(10)}
label_names = [label_map[i + 1] for i in range(10)]

# Confusion matrix
cm = confusion_matrix(y_true, y_pred, labels=list(range(1, 11)))
report = classification_report(y_true, y_pred, labels=list(range(1, 11)), target_names=[str(l) for l in label_names])

# Pretty-print
print("Confusion Matrix:")
print(pd.DataFrame(cm, index=[f"Actual {l}" for l in label_names],
                       columns=[f"Pred {l}" for l in label_names]))
print("\nClassification Report:")
print(report)

Confusion Matrix:
            Pred 10  Pred 20  Pred 30  Pred 40  Pred 50  Pred 60  Pred 70  \
Actual 10       212        0       30        0        5        0        0   
Actual 20         3        0        0        0        0        0        0   
Actual 30        33        0      120        0        3        1        0   
Actual 40         0        0        0        0        0        0        0   
Actual 50         4        0        8        0       31        0        0   
Actual 60         0        0        0        0        7        0        0   
Actual 70         0        0        0        0        0        0        0   
Actual 80         0        0        1        0        2        0        0   
Actual 90         1        0        1        0        1        0        0   
Actual 100        0        0        0        0        0        0        0   

            Pred 80  Pred 90  Pred 100  
Actual 10         0        0         0  
Actual 20         0        0         0  
Actual 30  

(5) Exercise: modify the code so that you feed into your random forest classifier that has been trained on 2020 data, on other years of Sentinel 2 data than 2020. Produce a publication quality figure that presents the following:
*   Landcover maps of Wellington for 2018, 2022 and 2024
*   The Test accuracy averages of the RF classifier for each iteration in terms of precision, recall and f1-score.
(15 pts)
_______________________________________________________________________________


In [22]:
####### Step 0: Define Variales
years = [2018, 2022, 2024]
cms = []
reports = []

# QUESTION: DOES THE REQUIREMENT ASK TO USE 2021 DATA???
# Prepare Ground Truth Data for model performance validation
# Load the available ESA WorldCover maps, only available for 2020 and 2021
worldcover_2020 = landcover_remapped
worldcover_2021 = ee.Image('ESA/WorldCover/v200/2021').select('Map').clip(aoi)

# Remap labels: create a new band with remapped values (10 → 1, ..., 100 → 10)
truth_2020_remapped = landcover_remapped
truth_2021_remapped = worldcover_2021.remap(valid_classes, remap_to).rename('Map_remapped')

# Create a mapping of years to truth maps, get the most recent one for each year
truth_maps = {
    2018: truth_2020_remapped,
    2022: truth_2021_remapped,
    2024: truth_2021_remapped
}

welly_images = []
welly_classified_list = []

for i, year in enumerate(years):
    ####### Step 1: get mask clouds in Sentinel-2 images of the year
    image = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(aoi)
        .filterDate(f'{year}-01-01', f'{year}-12-31')
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
        .map(mask_s2_clouds)
        .median()
        .clip(aoi))
    
    ####### Step 2: Apply classification to the year's image
    classified_image = image.select(bands).classify(classifier)

    ####### Step 3: Loop Through Each Year to Calculate and Print Accuracy Metrics
    
    # Combine the classified map and the ground truth map into a single image
    # We need to sample both at the same time for comparison
    validation_bands = classified_image.rename('classification').addBands(
        truth_maps[year].rename('ground_truth')
    )
    
    # Generate a new set of random points for validation
    # We use a large number of points to get a stable estimate of accuracy
    validation_sample = validation_bands.sample(
        region=aoi,
        scale=10,
        numPixels=10000,
        seed=42 # Use a seed for reproducibility
    )
    
    ####### Step 4: Pull data to client and run scikit-learn validation
    
    # Get predicted vs actual from validation set
    y_true, y_pred = fc_to_lists(validation_sample, 'ground_truth', 'classification')

    # Labels for original classes
    label_map = {i + 1: valid_classes.get(i).getInfo() for i in range(10)}
    label_names = [label_map[i + 1] for i in range(10)]

    # Generate the confusion matrix
    cm = confusion_matrix(y_true, y_pred, labels=list(range(1, 11)))
    
    
    # Generate the classification report
    report = classification_report(y_true, y_pred, labels=list(range(1, 11)), target_names=[str(l) for l in label_names])

    # Pretty-print the results using pandas for the confusion matrix
    cm_df = pd.DataFrame(cm, index=[f"Actual: {l}" for l in label_names],
                             columns=[f"Pred: {l}" for l in label_names])
    
    cms.append(cm_df)
    reports.append(report)


In [23]:
 ####### Step 3: Loop Through Each Year to Calculate and Print Accuracy Metrics
for year, cm_df, report in zip(years, cms, reports):
    print(f"\n--- {year} ---")
    print("\nConfusion Matrix:")
    print(cm_df)
    print("\nClassification Report:")
    print(report)


--- 2018 ---

Confusion Matrix:
             Pred: 10  Pred: 20  Pred: 30  Pred: 40  Pred: 50  Pred: 60  \
Actual: 10       1750         0       827         0        55         1   
Actual: 20          9         0         7         0         0         0   
Actual: 30        311         0      1378         0        63         2   
Actual: 40          1         0         2         0         0         0   
Actual: 50         58         0        46         0       367         3   
Actual: 60          4         0        10         0        55         1   
Actual: 70          0         0         0         0         0         0   
Actual: 80          5         0         2         0        59         0   
Actual: 90          2         0        11         0         4         0   
Actual: 100         0         0         0         0         0         0   

             Pred: 70  Pred: 80  Pred: 90  Pred: 100  
Actual: 10          0         0         0          0  
Actual: 20          0         0

# SVMs Applied to Satellite Data

In [24]:
# Define and train the SVM classifier
class_property = 'Map_remapped'
svm = ee.Classifier.libsvm(kernelType='RBF', gamma=0.5, cost=10).train(
    features=train,
    classProperty=class_property,
    inputProperties=bands
)

# Classify validation and test sets
val_classified = valid.classify(svm)
test_classified = test.classify(svm)

# Evaluate test performance
test_matrix = test_classified.errorMatrix(class_property, 'classification')
print("Confusion Matrix:")
print(pd.DataFrame(cm, index=[f"Actual {l}" for l in label_names],
                       columns=[f"Pred {l}" for l in label_names]))
print("\nClassification Report:")
print(report)

Confusion Matrix:
            Pred 10  Pred 20  Pred 30  Pred 40  Pred 50  Pred 60  Pred 70  \
Actual 10      2320        0      383        0       40        0        0   
Actual 20        24        0       38        0        0        0        0   
Actual 30       292        0     1223        0       50        1        0   
Actual 40         0        0        3        0        0        0        0   
Actual 50       100        0       40        0      378        1        0   
Actual 60         1        0        2        0       44        0        0   
Actual 70         0        0        0        0        0        0        0   
Actual 80         1        0        4        0       26        0        0   
Actual 90         4        0        6        0        1        0        0   
Actual 100        0        0        0        0        0        0        0   

            Pred 80  Pred 90  Pred 100  
Actual 10         4        1         0  
Actual 20         0        0         0  
Actual 30  

(6) Exercise: using an SVM model and the Landcare NZ 2024 landcover database, produce a landcover map of Great Barrier (Aotea) Island for 2025 (based off the Austral summer of 24/25).

Your map should be presented at a publication quality level with all the usual map components (scale, legend, north arrow, data attribution).

You will need to provide performance statistics of the model within your figure.

*   Here you can access the landcover database: https://lris.scinfo.org.nz/layer/104400-lcdb-v50-land-cover-database-version-50-mainland-new-zealand/. You will need to explore for yourself how to extract this data and then upload it to colab, then how to plug it into the SVM algorithim. I have provided some starter code below.

An intial workflow to get the data into the state you need it in to then use it as training data might look like:
- Download the ZIP manually from their browser, having set your area of interest and used the 'Export' tool top right.
- Upload it to Colab.
- Unzip it and load with GeoPandas.
(25 pts)



In [25]:
# Code to get you started
import zipfile
import geopandas as gpd

file_path = '../dataset/lris-lcdb-v50-land-cover-great-barrier-SHP/lcdb-v50-land-cover-database-version-50-mainland-new-zealand.shp'

# # Upload the ZIP manually using the Colab UI
# from google.colab import files
# uploaded = files.upload()  # <- Expects a ZIP

# # Unzip
# with zipfile.ZipFile("LCDB_v5.zip", 'r') as zip_ref: #<- Check file names
#     zip_ref.extractall("lcdb")

# Read shapefile
gdf = gpd.read_file(file_path)
print(gdf.head())


           Name_2018          Name_2012          Name_2008          Name_2001  \
0  Indigenous Forest  Indigenous Forest  Indigenous Forest  Indigenous Forest   
1     Sand or Gravel     Sand or Gravel     Sand or Gravel     Sand or Gravel   
2  Indigenous Forest  Indigenous Forest  Indigenous Forest  Indigenous Forest   
3     Sand or Gravel     Sand or Gravel     Sand or Gravel     Sand or Gravel   
4  Indigenous Forest  Indigenous Forest  Indigenous Forest  Indigenous Forest   

           Name_1996  Class_2018  Class_2012  Class_2008  Class_2001  \
0  Indigenous Forest          69          69          69          69   
1     Sand or Gravel          10          10          10          10   
2  Indigenous Forest          69          69          69          69   
3     Sand or Gravel          10          10          10          10   
4  Indigenous Forest          69          69          69          69   

   Class_1996  ... Wetland_96 Onshore_18 Onshore_12 Onshore_08 Onshore_01  \
0  

In [26]:
len(gdf)

605